# 构建高级有效的RAG

本次我们探索下，如何使用LlamaIndex构建一个有效且高效的RAG应用。

首先，我们来回顾下，什么是RAG?
> RAG（Retrieval-Augmented Generation，检索增强生成）是一种结合信息检索与自然语言生成的技术，旨在提升大型语言模型（LLM）在特定任务中的准确性和实用性。
其核心思想是：在生成回答时，模型不仅依赖自身的预训练知识，还会动态检索外部知识库中的相关信息，并将这些信息整合到最终输出中。

![Image](../../figures/RAG_define.png)

**RAG的典型流程**
1. 用户提问：例如，“2023年诺贝尔文学奖得主是谁？”。
2. 检索相关文档：从知识库中查找与“2023年诺贝尔文学奖”相关的最新信息。
3. 生成回答：模型结合检索到的信息（如获奖者姓名、背景等）生成答案，例如：“2023年诺贝尔文学奖授予了作家约翰·史密斯，以表彰他在当代文学中的杰出贡献。”

## RAG成功的要求
为了使agent可以回答有用又相关的问题，必须要求RAG系统有2个高级要求：
1. 检索组件必须能够找到与用户查询最相关的文档。
2. 生成组件必须能够有效利用检索到的文档，充分回答用户的查询。

## RAG适用典型的木桶理论场景
![Image](../../figures/RAG_mutong.png)


## 简单的RAG

In [ ]:
! pip install llama_index

> llama_index的使用依赖llm，默认使用open ai的gpt-3.5-turbo模型。因为国内使用openAI受限，所以这里我使用openAi like库使用我自己可访问的远程OPENAI API。这里我选择使用百炼api。

In [ ]:
! pip install llama-index-llms-openai-like

In [ ]:
! pip install llama-index-embeddings-dashscope

In [24]:
from llama_index.llms.openai_like import OpenAILike
import os

api_key = os.environ.get('DASHSCOPE_API_KEY')

llm = OpenAILike(
    model="qwen-max",
    api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=api_key,
    context_window=128000,
    is_chat_model=True,
    is_function_calling_model=False,
)

response = llm.complete("Hello World!")
print(str(response))

INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


Hello! It's great to meet you. Is there anything specific you'd like to know or discuss today?


In [25]:
from llama_index.core import SimpleDirectoryReader,Settings
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.embeddings.dashscope import DashScopeEmbedding
# Settings control global defaults
# LlamaIndex默认使用的Embedding模型被替换为百炼的Embedding模型
Settings.embed_model = DashScopeEmbedding(
    model_name="text-embedding-v2"
)
Settings.llm =OpenAILike(
    model="qwen-max",
    api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=api_key,
    context_window=128000,
    is_chat_model=True,
    is_function_calling_model=False,
)
# load data
input_data = ['d:/4px_work/MyNote/AI学习/知识库文档/诺贝尔/诺贝尔化学奖.pdf']
documents = SimpleDirectoryReader(input_files=input_data).load_data()
index = VectorStoreIndex.from_documents(documents=documents)
# 输出建立好的索引和压缩好的向量示例
print("输出向量化示例：")
for i, uuid in enumerate(index.vector_store.data.metadata_dict.keys()):
    print("文件名：", end='')
    print(index.vector_store.data.metadata_dict[uuid]['file_name'], end='')
    print("，文件大小：", index.vector_store.data.metadata_dict[uuid]['file_size'], end='')
    print("，文件类型：", index.vector_store.data.metadata_dict[uuid]['file_type'])
    print("压缩后向量：", end='')
    print(index.vector_store.data.embedding_dict[uuid][:3], '\n')
    if i > 3:
        break
query_engine = index.as_query_engine()
result = query_engine.query("请根据文档回答，1950年化学诺贝尔奖得主是谁？")
print(f"ai回答={result}")

输出向量化示例：
文件名：诺贝尔化学奖.pdf，文件大小： 424513，文件类型： application/pdf
压缩后向量：[-0.005259750033905756, -0.003843857630344754, 0.03567745992288473] 

文件名：诺贝尔化学奖.pdf，文件大小： 424513，文件类型： application/pdf
压缩后向量：[-0.0020895326978937773, -0.008882678964403387, 0.049723207929986205] 

文件名：诺贝尔化学奖.pdf，文件大小： 424513，文件类型： application/pdf
压缩后向量：[0.003513619834335137, -0.041321184015632284, 0.03971785713093928] 

文件名：诺贝尔化学奖.pdf，文件大小： 424513，文件类型： application/pdf
压缩后向量：[0.020978959848640722, -0.026000950237118424, 0.05086790264458059] 

文件名：诺贝尔化学奖.pdf，文件大小： 424513，文件类型： application/pdf
压缩后向量：[0.006048073393430071, -0.020876885707911096, 0.028967386131302116] 



INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


ai回答=文档中没有提供1950年诺贝尔化学奖得主的信息。


*可以看到，询问第一行的诺贝尔得奖可以得到正确答案，但是询问文档偏后的，比如2001年就无法获取结果，但文档中是有的。即使问同一个年份回答结果也很不稳定。*
**这是为什么呢？**

**多次回答结果示例：**
1. ai回答=文档中没有提供1972年诺贝尔化学奖得主的信息。
2. ai回答=1901年的诺贝尔化学奖得主是雅各布斯·亨里克斯·范托夫（Jacobus Henricus van’t Hoff, 1852–1911），他来自荷兰，获奖原因是“发现了化学动力学法则和溶液渗透压”。
3. ai回答=1908年的诺贝尔化学奖得主是欧内斯特·卢瑟福（Ernest Rutherford）。
4. ai回答=文档中没有提供1950年诺贝尔化学奖得主的信息。

## 高效的RAG-优化

> 在RAG高效检索技术中，控制系统效率和性能的一个关键参数是chunk_size，如何确定检索的最佳块大小？这就是 **LlamaIndex Response Evaluation**的用处。
- chunk太小，比如128，关键信息无法获取，尤其是similarity_top_k获取信息时。信息被切割到不同的chunk中，无法提供足够的上下文，导致llm理解不完整。
- chunk太大，比如512，噪音比较大，一次召回很多相似的内容，召回质量下降。且投喂给llm的上下文信息太多响应会更慢。
- 所以，评估最佳的chunk大小，在不牺牲速度的情况下捕获所有必要信息就很关键，一般向量召回，选择256-512之间进行测试，关键词召回就要大一些，比如1000-3000之间，可以保留更多关键词在一个chunk中。

### 使用LlamaIndex 分隔pdf文件，向量化存储

In [ ]:
!pip install llama-index

In [26]:
from llama_index.core.node_parser import (TextSplitter,SentenceSplitter)
from llama_index.core.schema import TextNode
TextNode?

Init signature:
TextNode(
    *args: Any,
    id_: str = <factory>,
    embedding: Optional[List[float]] = None,
    extra_info: Dict[str, Any] = <factory>,
    excluded_embed_metadata_keys: List[str] = <factory>,
    excluded_llm_metadata_keys: List[str] = <factory>,
    relationships: Dict[Annotated[llama_index.core.schema.NodeRelationship, PlainSerializer(func=<function <lambda> at 0x000001F4C55A6DE0>, return_type='str', when_used='always')], Union[llama_index.core.schema.RelatedNodeInfo, List[llama_index.core.schema.RelatedNodeInfo]]] = <factory>,
    metadata_template: str = '{key}: {value}',
    metadata_seperator: str = '\n',
    text: str = '',
    mimetype: str = 'text/plain',
    start_char_idx: Optional[int] = None,
    end_char_idx: Optional[int] = None,
    text_template: str = '{metadata_str}\n\n{content}',
) -> None
Docstring:     
Provided for backward compatibility.

Note: we keep the field with the typo "seperator" to maintain backward compatibility for
serialized obj

In [8]:
import nest_asyncio
nest_asyncio.apply()

from llama_index.llms.openai_like import OpenAILike
import os
from llama_index.core import (
    SimpleDirectoryReader,
    Settings,
    Response,
    VectorStoreIndex,
)
from llama_index.core.evaluation import (
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    EvaluationResult
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.dashscope import DashScopeEmbedding
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.readers.file import PDFReader
from llama_index.core.node_parser import (TextSplitter,SimpleNodeParser)
from llama_index.core.vector_stores import MetadataFilters, ExactMatchFilter
import pandas as pd
import logging
import re
import pdfplumber
from llama_index.core.schema import TextNode, Document

logging.basicConfig(level=logging.INFO)

# 设置API密钥
api_key = os.environ.get('DASHSCOPE_API_KEY')

# 配置全局设置
Settings.embed_model = DashScopeEmbedding(
    model_name="text-embedding-v2"
)
Settings.llm = OpenAILike(
    model="qwen-max",
    api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=api_key,
    context_window=128000,
    is_chat_model=True,
    is_function_calling_model=False,
)


**使用pdfplumber读取器提取带表格的pdf文件，做预处理（增加元数据，表格部分按行分隔）：**

In [ ]:
pip install pdfplumber

In [9]:
# -*- coding: utf-8 -*-
# !/usr/bin/env python3
import logging
import os
from typing import List

import pdfplumber
from llama_index.core.schema import TextNode, Document
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
import re


# PDF Loader
def format_record_line(kv_pairs_dict):
    # 将生硬的表格表头和字段的对应信息转为更流畅的自然语言表述方式：xxx年的诺贝尔化学奖获奖者是xxx,他们因为xxx而获奖
    year = kv_pairs_dict.get("年份",None)
    winner = kv_pairs_dict.get("获奖者",None)
    country = kv_pairs_dict.get("国籍",None)
    reason = kv_pairs_dict.get("获奖原因",None)
    negative_markers = ["未颁奖", "未颁发", "未授予", "空缺", "从缺"]
    formatted_text = ""
    awarded = not (winner is not None and any(marker in winner for marker in negative_markers))
    if year is not None:
        if awarded is False:
            formatted_text = f"{year}年的诺贝尔化学奖未颁发。"
        else:
            formatted_text = f"{year}年的诺贝尔化学奖获奖者是{winner}，因为{reason}而获奖。"
    return awarded,formatted_text


class NobelPdfLoader:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path
        self.logger = logging.getLogger(__name__)

    def load_pdf_with_reader(self):
        """使用pdfplumber逐页解析PDF，仅解析表格：第一行作为表头，其余作为记录。返回每页一个Document。"""
        documents: list[Document] = []
        try:
            with pdfplumber.open(self.pdf_path) as pdf:
                self.logger.info(f"pdfplumber 打开PDF，共 {len(pdf.pages)} 页")
                for page_index, page in enumerate(pdf.pages, start=1):
                    tables = page.extract_tables() or []

                    # 处理表格：取第一行作为表头，行到记录
                    for table_index, table in enumerate(tables, start=1):
                        # 如果 table 是空/None，或者 table 中所有行都是空的（没有一行是非空的），就进入这个 if 分支
                        if not table or not any(row for row in table):
                            continue
                        # 去除表格每一个值的前后空格，和无效表格数据
                        headers = [h.strip() if isinstance(h, str) else "" for h in (table[0] or [])]
                        headers = [h for h in headers if h is not None]
                        if not headers:
                            continue
                        for row in table[1:]:
                            if not row:
                                continue
                            cells = [c.strip() if isinstance(c, str) else "" for c in row]
                            kv_pairs_dict = {header: cells[idx].replace("\n", "") if idx < len(cells) else "" for idx, header in enumerate(headers)}
                            awarded, formatted_text = format_record_line(kv_pairs_dict)
                            documents.append(
                                Document(
                                    text=formatted_text,
                                    metadata={
                                        "page_number": page_index,
                                        "file_name": os.path.basename(self.pdf_path),
                                        "source": "pdfplumber",
                                        "year": kv_pairs_dict.get("年份", None),
                                        "winner": kv_pairs_dict.get("获奖者", None),
                                        "awarded": awarded
                                    },
                                )
                            )

                    # 只有在没有表格的情况下才提取页面文本,使用SentenceSplitter分块
                    if not tables:
                        page_text = page.extract_text() or ""
                        if page_text.strip():
                            splitter = SentenceSplitter(chunk_size=512, chunk_overlap=50)
                            page_texts = splitter.split_text(page_text.strip())
                            for trunk_text in page_texts:
                                documents.append(
                                    Document(
                                        text=trunk_text.strip(),
                                        metadata={
                                            "page_number": page_index,
                                            "file_name": os.path.basename(self.pdf_path),
                                            "source": "pdfplumber"
                                        },
                                    )
                                )

                    if len(documents) % 10 == 0:
                        self.logger.info(f"PDF第{documents[-1].metadata['page_number']}页，第{len(documents)}个chunk，内容为：{documents[-1]},metadata:{documents[-1].metadata}")

            self.logger.info(f"成功加载PDF文档，共生成 {len(documents)}个chunk。 ")
            return documents
        except Exception as e:
            self.logger.error(f"pdfplumber 解析失败: {e}")
            # 回退到SimpleDirectoryReader
            return SimpleDirectoryReader(input_files=[self.pdf_path]).load_data()



#### 使用chrome向量数据库，结合llama-index做基于年份元数据的检索

In [ ]:
pip install chromadb llama-index-vector-stores-chroma

In [21]:
# -*- coding: utf-8 -*-
# !/usr/bin/env python3
import logging
import chromadb
from llama_index.core import Document, VectorStoreIndex, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.vector_stores import MetadataFilters, ExactMatchFilter
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# 全局变量（可改为配置）
CHROMA_DB_DIR = "d:/4px_work/MyNote/AI学习/chroma_db"  # 本地持久化路径


class ChromaIndexManager:
    def __init__(self, collection_name):
       self.collection_name = collection_name
       self.logger = logging.getLogger(__name__)  # 使用模块级日志记录器

    def save_nodes(self, documents):
        """
        首次初始化
        :param documents:
        :return:
        """
        self.logger.info(f"开始保存文档到集合: {self.collection_name}")
        db = chromadb.PersistentClient(CHROMA_DB_DIR)
        chroma_collection = db.get_or_create_collection(self.collection_name)
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)
        VectorStoreIndex(documents, storage_context=storage_context, show_progress=True)  # 自动持久化
        self.logger.info("文档保存完成")

    def load_index(self):
        """
        加载索引
        :return:
        """
        self.logger.info(f"开始加载索引: {self.collection_name}")
        try:
            db = chromadb.PersistentClient(CHROMA_DB_DIR)
            chroma_collection = db.get_or_create_collection(self.collection_name)
            vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
            index= VectorStoreIndex.from_vector_store(vector_store)
            self.logger.info("索引加载完成")
            return index
        except Exception as e:
            self.logger.error(f"加载索引时出错: {e}")
            raise

    def query(self, question, index):
        """
        查询：先提取年份，用 metadata 过滤，再语义检索
        """
        # 假设你有这个函数来提取年份
        def _extract_year_from_question(question: str) -> int:
            import re
            # 匹配 1800~2099 之间的年份
            match = re.search(r'(18|19|20)\d{2}', question)
            return int(match.group()) if match else None

        year = _extract_year_from_question(question)
        # year = None
        self.logger.debug(f"提取年份: {year}")
        postprocessor = SimilarityPostprocessor(similarity_cutoff=0.6)

        if year is not None:
            self.logger.info(f" Filtering by year: {year}")
            filters = MetadataFilters(
                filters=[ExactMatchFilter(key="year", value=str(year))]
            )
            retriever = VectorIndexRetriever(
                index=index,
                similarity_top_k=3,
                filters=filters
            )
        else:
            self.logger.info("No year found, searching across all years")
            retriever = VectorIndexRetriever(
                index=index,
                similarity_top_k=3
            )
        # nodes = retriever.retrieve(question)
        # self.logger.info(f"检索到的节点数量:{len(nodes)}" )
        # for n in nodes:
        #     self.logger.info(f"Node text:{n.text}")
        #     self.logger.info(f"Node score:{n.score}")  # 看相似度分数

        engine = RetrieverQueryEngine(
            retriever=retriever,
            # node_postprocessors=[postprocessor]
        )
        response = engine.query(question)
        # 如果需要查看检索到的节点，可以通过 response 获取
        source_nodes = response.source_nodes
        self.logger.info(f"检索到的节点数量: {len(source_nodes)}")
        for n in source_nodes:
            self.logger.info(f"Node text:{n.text}")
            self.logger.info(f"Node score:{n.score}")  # 看相似度分数

        return response

## 最终整合测试：

In [22]:
import logging
logging.basicConfig(level=logging.INFO)

logger = logging.getLogger(__name__)
# logger.info("开始加载PDF文档...")
# pdf_path = "d:/4px_work/MyNote/AI学习/知识库文档/诺贝尔/诺贝尔化学奖.pdf"
# nobelPdfLoader = NobelPdfLoader(pdf_path)
# documents = nobelPdfLoader.load_pdf_with_reader()
# logger.info(documents[2].text)

# logger.info("加载PDF文档完成！")

# logger.info("开始构建索引...")
# indexManager = ChromaIndexManager(collection_name="chemical_nobel_prizes")
# indexManager = ChromaIndexManager(collection_name="chemical_nobel_prizes_v2")
# indexManager = ChromaIndexManager(collection_name="chemical_nobel_prizes_v3")
indexManager = ChromaIndexManager(collection_name="chemical_nobel_prizes_v4")
# 首次初始化-持久化保存
# indexManager.save_nodes(documents)
# 二次读取-读取持久化保存的节点
index = indexManager.load_index()

answer = indexManager.query("【2010】年诺贝尔化学奖的获得者是谁？", index)
logger.info(f"答案：{answer}\n\n")

answer = indexManager.query("【1916】年诺贝尔化学奖的获得者是谁？", index)
logger.info(f"答案：{answer}\n\n")

answer = indexManager.query("【1925】年诺贝尔化学奖的获得者是谁？", index)
logger.info(f"答案：{answer}\n\n")


INFO:__main__:开始加载索引: chemical_nobel_prizes_v4
INFO:__main__:索引加载完成
INFO:__main__: Filtering by year: 2010
INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:检索到的节点数量: 1
INFO:__main__:Node text:2010年的诺贝尔化学奖获奖者是理查德·赫克(Richard F. Heck, 1931–2015)根岸英一(Ei-ichi Negishi, 1935– )铃木章(Akira Suzuki, 1930– )，因为“对有机合成中钯催化交叉偶联反应的研究”“for palladium-catalysed cross couplings in organic synthesis”而获奖。
INFO:__main__:Node score:0.6184732073652592
INFO:__main__:答案：2010年诺贝尔化学奖的获得者是理查德·赫克（Richard F. Heck, 1931–2015）、根岸英一（Ei-ichi Negishi, 1935–）和铃木章（Akira Suzuki, 1930–）。他们因为“对有机合成中钯催化交叉偶联反应的研究”而获奖。


INFO:__main__: Filtering by year: 1916
INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:检索到的节点数量: 1
INFO:__main__:Node text:1916年的诺贝尔化学奖未颁发。
INFO:__main__:Node score:0.5785996608485441
INFO:__main__:答案：1916年的诺贝尔化学奖没有颁发。


INFO:__main__: Filtering by year: 1

#### 测试filter集合查询是否生效：

In [19]:
import chromadb

# CHROMA_DB_DIR = "d:/4px_work/MyNote/AI学习/chroma_db"        # 本地持久化路径
COLLECTION_NAME = "chemical_nobel_prizes_v4"    # 集合名
# 连接到你的数据库
client = chromadb.PersistentClient(path=CHROMA_DB_DIR)
collection = client.get_collection(COLLECTION_NAME)

# 直接用 Chroma 的 where 查询
results = collection.get(
    where={"year": "1925"}
)

print("Chroma where 查询结果数量:", len(results["ids"]))
if results["metadatas"]:
    print("第一个结果的 metadata:", results["metadatas"][0])
    print("第一个结果的 documents:", results['documents'][0])


Chroma where 查询结果数量: 1
第一个结果的 metadata: {'_node_content': '{"id_": "2ba8ad85-2bf0-4a65-8ff6-f584d480019b", "embedding": null, "metadata": {"page_number": 4, "file_name": "\\u8bfa\\u8d1d\\u5c14\\u5316\\u5b66\\u5956.pdf", "source": "pdfplumber", "year": "1925", "winner": "\\u91cc\\u590f\\u5fb7\\u00b7\\u963f\\u9053\\u592b\\u00b7\\u5e2d\\u683c\\u8499\\u8fea(Richard Adolf Zsigmondy, 1865\\u20131929)", "awarded": true}, "excluded_embed_metadata_keys": [], "excluded_llm_metadata_keys": [], "relationships": {}, "metadata_template": "{key}: {value}", "metadata_separator": "\\n", "image_resource": null, "audio_resource": null, "video_resource": null, "text_template": "{metadata_str}\\n\\n{content}", "class_name": "Document", "text": ""}', '_node_type': 'Document', 'awarded': True, 'doc_id': 'None', 'document_id': 'None', 'file_name': '诺贝尔化学奖.pdf', 'page_number': 4, 'ref_doc_id': 'None', 'source': 'pdfplumber', 'winner': '里夏德·阿道夫·席格蒙迪(Richard Adolf Zsigmondy, 1865–1929)', 'year': '1925'}
第一个结果的 d

**上面经历了，pdf表格的抽取转换为文本格式，增加metadata元数据检索技术，保存到chrome轻量化向量数据库中，提取年份关键字增
强检索能力，提高检索效率。最终成功的检索到诺贝尔得奖的数据！**

**这中间经历了很多失败，又不断的优化完善,花了四五天的时间，实属不易。这里总结记录下问题以及解决办法：**
1. 最开始使用`SimpleDirectoryReader`解析pdf文件，提问，根据相似度返回的结果，大部分时候都返回是“未颁奖”的结果，年份不对。
> 思考原因，认为可能是pdf的解析text按页返回的表格形式，第一行是表头，表头和内容是分离的，且表格一个cell中很多内容有换行符，影响了chunk切割效果。于是改为使用`pdfplumber`提取表格内容表头等，做表格数据的预处理，做到一条获奖记录完整的整理为：“记录：表头key：对应值value”这样的形式，比如：记录: 年份: 1930 | 获奖者: 汉斯·费歇尔 | 国籍: 德国 | 获奖原因: 对血红素..。但是转换后再检索，依然效果不理想。

2. 然后问ai，告诉我或许增强年份关键字，提取年份关键字，增加year和是否颁奖的元数据`metadata`，检索时，先过滤元数据`retriever.search(query, filter={"year": 1930})`
> 增加以上`MetadataFilters`后，直接没有任何结果返回了。又苦苦探寻很久，加各种代码调试，增加检查点，确认是否metadata中包含年份。后ai告诉我，`VectorStoreIndex` 默认使用的是：`SimpleNodeStore`（存储节点） `InMemoryVectorStore`（存储向量）,但这两个组件：不支持元数据过滤,`MetadataFilters` 会被静默忽略（不会报错，但也不生效）。
> 所以我改为使用Chrome本地向量数据库检索过滤，即`NobelPrizeRetriever`。

3. 增加filter和chrome本地向量库改写后，仍然无法返回结果了，都是`Empty Response`。
> 最后根据ai的提示，一步步排查，metadata没错，存储检索能查到，猜测最有可能是后处理器影响了。
> 也就是：`postprocessor = SimilarityPostprocessor(similarity_cutoff=0.7)`
> 即使 Chroma 返回了 1 个结果，如果它的相似度 < 0.7，就会被后处理器丢弃，导致 Empty Response。

> 根据提示，我先去除postprocessor，再测试，奇迹般出结果了。太棒了！
> 后来写代码测试，我发现正确答案的相似度只有0.42左右，难怪加上`postprocessor`后无法返回结果。

#### 最后结合这个结果，我进一步问了AI为什么这样，以及chrome的初始化及复用问题，这里记录下：
#### 问题是这样的：
你说对了，我去除`postprocessor = SimilarityPostprocessor(similarity_cutoff=0.7)`后可以正常返回结果了，太棒了！
1. 我看正确结果的相似度分数只有0.42，这是为什么呢？
2. 我使用了Chroma 存储解析后包含年份的元数据。我想知道每次我重新执行NobelPrizeRetriever()初始化，并build_index后，我的本地索引是一直再追加的吗？有没有更好的办法帮我自动检测并持续使用？或真实生产环境下，索引存储是怎么使用的呢？何时初始化怎么复用以及清理的时机是怎样的呢？
#### AI回答是这样的，个人认为很棒，总结的很好，这里记录下：

太棒了！你已经成功解决了核心问题 👏  
现在我们来深入回答你的两个非常专业、贴近生产实践的问题：

---

## ✅ 问题 1：为什么正确结果的相似度分数只有 `0.42`？

### 🔍 原因分析

相似度分数低（如 `0.42`）说明：**查询文本和文档文本在向量空间中的语义距离较远**，即使它们“逻辑上”是匹配的。

这在 RAG 系统中非常常见，原因如下：

---

### 🧩 原因 1：**文本表达方式差异大（语义鸿沟）**

| 查询 | 文档内容 |
|------|----------|
| `"1901年诺贝尔化学奖是谁？"` | `"记录: 年份: 1901 | 获奖者: 雅各布斯·亨里克斯·范托夫..."` |

虽然人类一看就知道匹配，但向量模型看到的是：

- 查询是“问句 + 年份”
- 文档是“结构化记录”，没有“是谁”这样的疑问结构

→ 模型认为它们“不像”

---

### 🧩 原因 2：**使用的 embedding 模型对中文支持一般**

你可能用的是默认的 `BAAI/bge-small-zh` 或 `text-embedding-ada-002`，这些模型：

- 对中文支持尚可，但不如英文
- 对“结构化文本 vs 自然语言问句”的对齐能力弱
- 容易导致“关键词匹配但语义分低”

---

### 🧩 原因 3：**文档中缺乏“问题式表达”**

你的文档是：
```
记录: 年份: 1901 | 获奖者: ... | 国籍: ... | 获奖原因: ...
```

它没有说：
> “1901 年诺贝尔化学奖得主是范托夫”

→ 缺少“主谓宾”完整句子，embedding 模型难以建立强语义关联

---

### ✅ 解决方案

#### ✅ 方案 1：**优化文档文本格式（推荐）**

将结构化数据转为自然语言描述，提升语义匹配度：

```python
text = (
    f"{year}年诺贝尔化学奖由{winner}获得，"
    f"他是{nationality}人，"
    f"获奖原因是{reason}。"
)
```

例如：
> “1901年诺贝尔化学奖由雅各布斯·亨里克斯·范托夫获得，他是荷兰人，获奖原因是发现了化学动力学规律。”

这样更容易与“1901年诺贝尔化学奖是谁”匹配。

---

#### ✅ 方案 2：**使用更强的中文 embedding 模型**

推荐替换默认模型为：

- `BAAI/bge-large-zh-v1.5`（本地）
- `text-embedding-3-large`（OpenAI）
- `m3e-large`（中文专用）

```python
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-zh-v1.5")
Settings.embed_model = embed_model
```

---

#### ✅ 方案 3：**降低 `similarity_cutoff` 或去掉后处理器**

既然你知道 `0.42` 是正确结果，就不要一刀切地过滤掉。

```python
# 更宽松
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.4)
```

或：

```python
# 不过滤，由 LLM 最终判断
# 去掉 postprocessor，让所有候选都进入 prompt
```

---

#### ✅ 方案 4：**使用重排序模型（Reranker）**

在向量检索后，用一个更强的模型对 top-k 结果重排序：

```python
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank

reranker = RankGPTRerank(top_n=3)
engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[reranker]  # ← 用语义更强的模型重排序
)
```

---

## ✅ 问题 2：索引是追加的吗？如何复用、管理、清理？

### 🎯 核心答案

> ❌ **不是追加的！每次 `build_index` 都会覆盖同名集合（collection）**，除非你手动控制。

我们来详细拆解生产环境下的最佳实践。

---

## 🧱 1. Chroma 的集合（Collection）行为

```python
collection = client.get_or_create_collection("chemical_nobel_prizes")
```

- 如果集合不存在 → 创建
- 如果存在 → 返回已有集合
- **但 `VectorStoreIndex` 默认会“增量添加”节点，不会自动去重**

→ 所以你每次运行 `build_index`，**如果不清理，就会重复插入相同数据**

---

## ✅ 生产环境最佳实践

### ✅ 实践 1：**索引只构建一次，后续复用**

```python
class NobelPrizeRetriever:
    def __init__(self, db_path="./chroma_db", collection_name="nobel"):
        self.client = chromadb.PersistentClient(path=db_path)
        self.collection = self.client.get_or_create_collection(collection_name)
        
        # 检查是否已有数据
        if self.collection.count() == 0:
            print("首次运行，构建索引...")
            self.build_index()
        else:
            print("索引已存在，加载中...")
            self.load_index()
```

---

### ✅ 实践 2：**如何“更新”索引？**

#### 场景：PDF 更新了，新增了 1950 年记录

```python
# 方案：只插入新数据
new_nodes = [new_node_for_1950]

# 直接插入，不重建
index.insert_nodes(new_nodes)
```

✅ **增量更新，不重复插入旧数据**

---

### ✅ 实践 3：**如何“重新构建”索引？**

当结构变化（如 metadata 字段变更），需要清空重建：

```python
# 删除旧集合
client.delete_collection("chemical_nobel_prizes")

# 重建
collection = client.get_or_create_collection("chemical_nobel_prizes")
index = VectorStoreIndex(nodes, storage_context=storage_context)
```

---

### ✅ 实践 4：**生产环境索引管理策略**

| 场景 | 策略 |
|------|------|
| 数据不变 | 构建一次，永久复用 |
| 数据少量更新 | `insert_nodes()` 增量添加 |
| 数据结构变更 | 删除集合，重建索引 |
| 定期全量更新 | 脚本化：`delete + build` |
| 多版本管理 | 用不同 `collection_name`（如 `nobel_v1`, `nobel_v2`） |

---

### ✅ 实践 5：**自动化检测是否需要重建**

```python
def should_rebuild_index(self, current_hash):
    """根据文件哈希判断是否需要重建"""
    try:
        with open("index_metadata.json", "r") as f:
            meta = json.load(f)
        return meta.get("file_hash") != current_hash
    except:
        return True

def save_index_metadata(self, file_hash):
    with open("index_metadata.json", "w") as f:
        json.dump({"file_hash": file_hash, "build_time": time.time()}, f)
```

---

## ✅ 总结

### ✅ 问题 1：为什么相似度只有 0.42？
- 因为“问句”和“结构化记录”语义差异大
- 解决方案：
  - 优化文档为自然语言
  - 换更强的 embedding 模型
  - 降低 `similarity_cutoff`
  - 使用 reranker

### ✅ 问题 2：索引如何管理？
- ❌ 不是自动追加，重复运行会重复插入
- ✅ 正确做法：
  - 首次构建，后续复用
  - 增量更新用 `insert_nodes`
  - 变更结构时重建
  - 用文件哈希自动判断是否需要重建

---

## 🏁 最终建议

```python
# 1. 把文档转为自然语言描述
# 2. 使用 bge-large-zh 嵌入模型
# 3. 去掉 similarity_cutoff 或设为 0.4
# 4. 索引只构建一次，或用哈希检测更新
# 5. 生产环境用 collection 版本管理
```

## 以上参考的文档
* llama_index:
    * [quickstart](https://docs.llamaindex.ai/en/stable/#30-second-quickstart)
    * [openai_like llm](https://docs.llamaindex.ai/en/stable/api_reference/llms/openai_like/)
* RAG:
    * [构建高级 RAG 的指南和技巧](https://baoyu.io/translations/rag/a-cheat-sheet-and-some-recipes-for-building-advanced-rag)
      
* llama_index with 百炼：
    * [llama_index with 百炼](https://help.aliyun.com/zh/model-studio/dashscopellm-in-llamaindex)
      
* [LlamaIndex Chunk Size Optimization Recipe](https://www.llamaindex.ai/blog/evaluating-the-ideal-chunk-size-for-a-rag-system-using-llamaindex-6207e5d3fec5)
* [Evaluating the Ideal Chunk Size for a RAG System using LlamaIndex](https://colab.research.google.com/drive/1LPvJyEON6btMpubYdwySfNs0FuNR9nza?usp=sharing)
* [RagEvaluatorPack](https://llamahub.ai/l/llama-packs/llama-index-packs-rag-evaluator?from=)
  

                  